In [1]:
%pip install datasets
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

dataset = load_dataset("commonsense_qa")

print(dataset)


c:\nlp_1\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1221
    })
    test: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1140
    })
})


In [3]:
sample = dataset["train"][0]
print(sample)

{'id': '075e483d21c29a511267ef62bedc0461', 'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?', 'question_concept': 'punishing', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']}, 'answerKey': 'A'}


In [4]:
sample = dataset["validation"][0]
print(sample)

{'id': '1afa02df02c908a558b4036e80242fac', 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?', 'question_concept': 'revolving door', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['bank', 'library', 'department store', 'mall', 'new york']}, 'answerKey': 'A'}


In [5]:
sample = dataset["test"][0]
print(sample)

{'id': '90b30172e645ff91f7171a048582eb8b', 'question': 'The townhouse was a hard sell for the realtor, it was right next to a high rise what?', 'question_concept': 'townhouse', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['suburban development', 'apartment building', 'bus stop', 'michigan', 'suburbs']}, 'answerKey': ''}


In [6]:
import pandas as pd

df_train = pd.DataFrame(dataset["train"])
df_val = pd.DataFrame(dataset["validation"])
df_test = pd.DataFrame(dataset["test"])

print(f"Train Size: {len(df_train)}")
print(f"Validation Size: {len(df_val)}")
print(f"Test Size: {len(df_test)}")

print(df_train.head())


Train Size: 9741
Validation Size: 1221
Test Size: 1140
                                 id  \
0  075e483d21c29a511267ef62bedc0461   
1  61fe6e879ff18686d7552425a36344c8   
2  4c1cb0e95b99f72d55c068ba0255c54d   
3  02e821a3e53cb320790950aab4489e85   
4  23505889b94e880c3e89cff4ba119860   

                                            question question_concept  \
0  The sanctions against the school were a punish...        punishing   
1  Sammy wanted to go to where the people were.  ...           people   
2  To locate a choker not located in a jewelry bo...           choker   
3  Google Maps and other highway and street GPS s...          highway   
4  The fox walked from the city into the forest, ...              fox   

                                             choices answerKey  
0  {'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...         A  
1  {'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...         B  
2  {'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...         A  
3  {'label'